In [23]:
from torch.utils.data import Dataset
import os
import matplotlib.pyplot as plt
import matplotlib.image as Mapping
import numpy as np
from PIL  import Image
import cs231n.datasets.Load_faces as dst
from torch.utils.data import DataLoader
import torchvision.transforms as T
import torch.nn as nn
import torch
import matplotlib.pyplot as plt
from torch import optim
from torch.utils.checkpoint import checkpoint_sequential
from sklearn.preprocessing import MinMaxScaler
from torch.utils.tensorboard import SummaryWriter
import torchvision

In [24]:
dir_path = "C:/Users/wtser/Desktop/learnData/data/faces/"
transform = T.Compose([T.Scale(64),T.ToTensor(),T.Normalize([0.6756,0.5782,0.5534],[0.2733,0.2703,0.2562])])
train_data = dst.load_Face(root =dir_path,number = 2000,transform=transform)
device = torch.device('cuda')
writer = SummaryWriter(log_dir="E:/GANPIC/second/")

D:\Anaconda\lib\site-packages\torchvision\transforms\transforms.py:208: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +


In [25]:
def normData(loader_data):
    for x,y in loader_data:
        x = x/2.4720
        yield (x,y)
    

In [26]:
loader_train = DataLoader(train_data,batch_size=32,shuffle=True)
# loader_train = normData(loader_train)

In [27]:
class SimpleCustomBatch:
    def __init__(self, data):
        transposed_data = list(zip(*data))
        print(transposed_data[1])
        self.inp = torch.stack(transposed_data[0], 0)
        self.tgt = torch.stack(transposed_data[1], 0)

    def pin_memory(self):
        self.inp = self.inp.pin_memory()
        self.tgt = self.tgt.pin_memory()
        return self

def collate_wrapper(batch):
    return SimpleCustomBatch(batch)


In [28]:
class Reshape(nn.Module):
    def __init__(self,_shape):
        super(Reshape,self).__init__()
        self.shape = _shape
    
    def forward(self,x):
        return x.view(*self.shape)

In [29]:
class GAN(nn.Module):
    def __init__(self,indim,img_shape):
        super(GAN,self).__init__()
        self.img_shape = img_shape
        
        def block(indim,outdim,normalize=True):
            layers = [nn.Linear(indim,outdim)]
            if normalize:
                layers.append(nn.BatchNorm1d(outdim,0.8))
            layers.append(nn.LeakyReLU(0.2,inplace=False))
            return layers
        
        self.model = nn.Sequential(nn.ConvTranspose2d(100,512,kernel_size=4,stride=1,padding=0,bias=False),
                                   nn.ReLU(inplace=True),
                                   nn.ConvTranspose2d(512,256,kernel_size=4,stride=2,padding=1,bias=False),
                                   nn.BatchNorm2d(256),
                                   nn.ReLU(inplace=True),
                                   nn.ConvTranspose2d(256,128,kernel_size=4,stride=2,padding=1,bias=False),
                                   nn.BatchNorm2d(128),
                                   nn.ReLU(inplace=True),
                                   nn.ConvTranspose2d(128,64,kernel_size=4,stride=2,padding=1,bias=False),
                                   nn.BatchNorm2d(64),
                                   nn.ReLU(inplace=True),
                                   nn.ConvTranspose2d(64,3,kernel_size=4,stride=2,padding=1,bias=False),
                                   nn.Tanh()
                                  )
    def forward(self,x):
        out = self.model(x)
        return out

In [30]:
class Discriminator(nn.Module):
    def __init__(self,img_shape):
        super(Discriminator,self).__init__()
        self.model = nn.Sequential(nn.Conv2d(3,64,kernel_size=4,stride=2,padding=1,bias=False),
                                  nn.LeakyReLU(0.2,inplace=True),
                                  nn.Conv2d(64,128,kernel_size=4,stride=2,padding=1,bias=False),
                                  nn.BatchNorm2d(128),
                                  nn.LeakyReLU(0.2,inplace=True),
                                  nn.Conv2d(128,256,kernel_size=4,stride=2,padding=1,bias=False),
                                  nn.BatchNorm2d(256),
                                  nn.LeakyReLU(0.2,inplace=True),
                                  nn.Conv2d(256,512,kernel_size=4,stride=2,padding=1,bias=False),
                                  nn.BatchNorm2d(512),
                                  nn.LeakyReLU(0.2,inplace=True),
                                  nn.Conv2d(512,1,kernel_size=4,stride=1,padding=0,bias=False),
                                   nn.Sigmoid()
                                  )
        
    def forward(self,x):
        out = self.model(x)
        return out.view(-1,1)

In [31]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [32]:
def save_model_dict(model,path):
    torch.save(model.state_dict(),path)

In [33]:
def load_model_dict(model,path):
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint)
    return model

In [34]:
generator = GAN(100,(3,96,96))
generator.apply(weights_init)
generator = load_model_dict(generator,"E:/GANModel/DCGAN/generator/generator_epoch2000")
generator = generator.cuda()

In [35]:
discriminator = Discriminator([3,96,96])
discriminator.apply(weights_init)
discriminator = load_model_dict(discriminator,"E:/GANModel/DCGAN/discriminator/discriminator_epoch2000")
discriminator = discriminator.cuda()

In [36]:
epochs = 1
k = 3
criterion = nn.BCELoss()
# Optimizers
optimizer_G = optim.Adam(generator.parameters(),lr = 2e-4,betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=2e-4,betas=(0.5, 0.999))
# optimizer_G = optim.Adam(generator.parameters(),lr = 2e-4)
# optimizer_D = optim.Adam(discriminator.parameters(), lr=2e-4)
generator_path = "E:/GANModel/DCGAN/generator/"
discriminator_path = "E:/GANModel/DCGAN/discriminator/"

In [ ]:
for i in range(2001,2201):
    for index,(x,y) in enumerate(loader_train):
        fake = torch.zeros(x.shape[0],1,requires_grad=False).to(device)
        real = torch.rand(x.shape[0],1,requires_grad=False).to(device)
        x = x /2.4720
        for j in range(3):
            optimizer_D.zero_grad()
            real_data = x.to(device)
            output = discriminator(real_data)
            gloss_d_real = criterion(output,real).to(device)
            gloss_d_real.backward()
        
            noise = torch.randn(x.shape[0],100,1,1,requires_grad=False).to(device)
            fake_img = generator(noise)
            output = discriminator(fake_img.detach())
            gloss_d_fake = criterion(output,fake).to(device)
            gloss_d_fake.backward()
            gloss_d = (gloss_d_fake + gloss_d_real)/2
            optimizer_D.step()
        
        optimizer_G.zero_grad()
        output = discriminator(fake_img)
        gloss_g = criterion(output,real).to(device)
        gloss_g.backward()
        optimizer_G.step()
    writer.add_scalars('loss',{'gloss_g':gloss_g,'gloss_d':gloss_d,'gloss_d_real':gloss_d_real,
                              'gloss_d_fake':gloss_d_fake},global_step=i)
    
    if i%100 == 0:
            print('gloass_g:{},gloss_d:{},gloss_d_real{},gloss_d_fake{}'.format(gloss_g,gloss_d,gloss_d_real,gloss_d_fake))
            store_img = torchvision.utils.make_grid((fake_img+1)/2.0,padding=2)
            writer.add_image('generate_img',store_img,global_step=i)
            
            g_path = os.path.join(generator_path,"generator_epoch{}".format(i))
            d_path = os.path.join(discriminator_path,"discriminator_epoch{}".format(i))
            save_model_dict(generator,g_path)
            save_model_dict(discriminator,d_path)
#             a = fake_img[0].cpu()
#             img_numpy = a.detach().numpy()
#             img_numpy = img_numpy.transpose(1,2,0)
#             print(img_numpy[:2,:2])
#             plt.imshow(((img_numpy+1.0)/2.0))
#             plt.show()
            writer.close()

In [19]:
?writer.add_scalars()